# Import and Data loading

In [ ]:
import numpy as np
import pandas as pd
import random as rnd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



#data loading
data = np.load("C:/Users/gabri/Desktop/uni/tesi/csgo_archive/legit/legit.npy")
data_cheater = np.load("C:/Users/gabri/Desktop/uni/tesi/csgo_archive/cheaters/cheaters.npy")




## setting the device

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Data preprocessing 



## Concatenation, adding labels, splitting in training and test, standarizing the data

In [ ]:
total = np.concatenate((data, data_cheater), axis=0)
labels_legit = np.zeros(len(data))
labels_cheater=np.ones(len(data_cheater))
labels = np.concatenate((labels_legit, labels_cheater))

#reshape dei dati per la SVM
X_train, X_test, y_train, y_test = train_test_split(total, labels, test_size=0.20, random_state=42, stratify=labels)

#rescaling dei dati
scaler = StandardScaler()
X_train_scaled = X_train
X_test_scaled = X_test

X_train_small = X_train_scaled[:1000, :10, 96:192]
y_train_small = y_train[:1000]
X_test_small = X_test_scaled[:200, :10, 96:192]
y_test_small = y_test[:200]

print(X_train_small.shape)


## transformation of data into tensor format

In [ ]:
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).permute(0, 3, 1, 2)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).permute(0, 3, 1, 2)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# CNN classifier

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class CNNClassifier(nn.Module):
    #initialization of the convolutional network defining the different layers
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(30, 64, kernel_size=(3, 3), stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(3, 3), stride=1, padding=1)
        self.fc1 = nn.Linear(6144, 256)
        self.fc2 = nn.Linear(256, 1)
        self.sigmoid = nn.Sigmoid()
    
    #activation function between layers
    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))
        x = nn.MaxPool2d(kernel_size=(2, 2))(x)
        x = nn.ReLU()(self.conv2(x))
        x = nn.MaxPool2d(kernel_size=(2, 2))(x)
        x = x.view(x.size(0), -1)
        x = nn.ReLU()(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

model = CNNClassifier().to(device)

# Loss function and optimization
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training cycle
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        
        inputs = inputs.permute(0, 2, 3, 1).contiguous()
        outputs = model(inputs)
        labels = labels.view(-1, 1)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / (i + 1):.4f}')
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

# Model evaluation
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs = inputs.permute(0, 2, 3, 1).contiguous()
            outputs = model(inputs)
            predicted = (outputs > 0.5).float()
            labels = labels.view(-1, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f'Test Accuracy: {100 * correct / total:.2f}%')

# LSTM model classifier

In [ ]:
X_train_tensor = torch.tensor(X_train_small.reshape((X_train_small.shape[0], -1, 5)), dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train_small, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test_small.reshape((X_test_small.shape[0], -1, 5)), dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test_small, dtype=torch.float32).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Creation of LSTM 
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

input_dim = 5
hidden_dim = 128
layer_dim = 2
output_dim = 1

model = LSTMClassifier(input_dim, hidden_dim, layer_dim, output_dim).to(device)

# Loss function and optimization
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training cycle
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        
        outputs = model(inputs)
        labels = labels.view(-1, 1)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / (i + 1):.4f}')
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

# Model evaluation
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            outputs = model(inputs)
            predicted = (outputs > 0.5).float()
            labels = labels.view(-1, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f'Test Accuracy: {100 * correct / total:.2f}%')

# Key feature dataset

 Creation of dataset containing important key feature of the entire dataset such as mean, variance for all the data and for only the cheaters or the legit players

In [ ]:
medie = np.mean(data, axis=(2))
varianza = np.mean(data, axis=(2))
medie_cheaters = np.mean(data_cheater, axis=(2))
varianza_cheaters = np.mean(data_cheater, axis=(2))

print(medie)
print(medie[0])
#concatenazione 
result=np.stack((medie,varianza), axis=1)
result_cheater=np.stack((medie_cheaters,varianza_cheaters), axis=1)
total = np.concatenate((result, result_cheater), axis=0)

labels_legit = np.zeros(len(result))
labels_cheater=np.ones(len(result_cheater))
labels = np.concatenate((labels_legit, labels_cheater))

#reshape dei dati per la SVM
X = total.reshape(12000, -1)
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.20, random_state=42, stratify=labels)

#rescaling dei dati
scaler = StandardScaler()
X_train_scaled = X_train
X_test_scaled = X_test

print(X_train_scaled.shape[1])


# MLP model

In [ ]:
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).view(-1,1,X_test_scaled.shape[1])
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).view(-1,1,X_test_scaled.shape[1])
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(X_train_tensor.shape)
print(y_train_tensor.shape)
def get_accuracy(logit, target):
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects / target.size(0)
    return accuracy.item()

def compute_weight_norm(model):
    norm = 0.0
    for name, param in model.named_parameters():
      if "weight" in name:
        norm += torch.norm(param.data, p= 2)
    return norm.cpu().item()

def train_model(model, num_epochs, trainloader, criterion, optimizer):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


    model = model.to(device)

    losses = []
    accs = []

    norms = []

    for epoch in range(num_epochs):
        train_running_loss = 0.0
        train_acc = 0.0

        
        model.train()
        ## training step
        for i, (inputs, labels) in enumerate(trainloader):

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            # forward + backprop + loss
            logits = model(inputs)
            loss = criterion(logits, labels)

            # Reset the gradients to zero

            loss.backward()

            # update model params
            optimizer.step()

            train_running_loss += loss.item()
            train_acc += get_accuracy(logits, labels)


        losses.append(train_running_loss / i)
        accs.append(train_acc/i)

        norms.append(compute_weight_norm(model))
        model.eval()
        print(f"Epoch: {epoch+1} | Loss: {train_running_loss / i:.4f} | Train Accuracy: {train_acc/i:.4f}")

    return losses, accs, norms

#creation of layers and activation function for MLP
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=6, kernel_size=10, padding=1)
        self.dropout1 = nn.Dropout(p=0.20)

        self.conv2 = nn.Conv1d(in_channels=6, out_channels=18, kernel_size=10, padding=1)
        self.dropout2 = nn.Dropout(p=0.20)


        self.conv3 = nn.Conv1d(in_channels=18, out_channels=30, kernel_size=10, padding=1)
        self.dropout3 = nn.Dropout(p=0.20)

        self.flattern = nn.Flatten()

        self.dropout4 = nn.Dropout(p=0.20)


        self.fc1 = nn.Linear(8370, 1000)

        self.fc2 = nn.Linear(1000, 500)

        self.fc3 = nn.Linear(500,64)

        self.fc4 = nn.Linear(64, 2)



    def forward(self, x):


        x = F.relu(self.conv1(x))
        x = self.dropout1(x)

        x = F.relu(self.conv2(x))
        x = self.dropout2(x)

        x = F.relu(self.conv3(x))
        x = self.dropout3(x)

        x = x.flatten(start_dim = 1)

        x = torch.relu(self.fc1(x))

        x = torch.relu(self.fc2(x))

        x = torch.relu(self.fc3(x))

        x = (self.fc4(x))
        return x





In [ ]:
input_size = X_train.shape[1]
output_size = 1

model = MLP()

#Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

_, _, W_adam = train_model(model, 100, train_loader, criterion, optimizer)


In [ ]:
from sklearn.ensemble import IsolationForest

modelisolationforest = IsolationForest(contamination=0.18, random_state=42)
modelisolationforest.fit(X_train_scaled)

y_train_pred_if = modelisolationforest.predict(X_train_scaled)
y_test_pred_if = modelisolationforest.predict(X_test_scaled)

y_train_pred_if = np.where(y_train_pred_if == -1,1,0)
y_test_pred_if = np.where(y_test_pred_if == -1,1,0)

train_accuracy_if = accuracy_score(y_train,y_train_pred_if)
test_accuracy_if = accuracy_score(y_test,y_test_pred_if)

print(train_accuracy_if)
print(test_accuracy_if)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

medie = np.mean(data, axis=(2))
varianza = np.mean(data, axis=(2))
medie_cheaters = np.mean(data_cheater, axis=(2))
varianza_cheaters = np.mean(data_cheater, axis=(2))

print(medie)
print(medie[0])
#concatenazione 
result=np.stack((medie,varianza), axis=1)
result_cheater=np.stack((medie_cheaters,varianza_cheaters), axis=1)
total = np.concatenate((result, result_cheater), axis=0)

labels_legit = np.zeros(len(result))
labels_cheater=np.ones(len(result_cheater))
labels = np.concatenate((labels_legit, labels_cheater))

#reshape dei dati per la SVM
X = total.reshape(12000, -1)
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.20, random_state=42, stratify=labels)

#rescaling dei dati
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

print(X_train_scaled.shape[1])

In [ ]:
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(X_train_tensor.shape)
print(y_train_tensor.shape)
def get_accuracy(logit, target):
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects / target.size(0)
    return accuracy.item()

def compute_weight_norm(model):
    norm = 0.0
    for name, param in model.named_parameters():
      if "weight" in name:
        norm += torch.norm(param.data, p= 2)
    return norm.cpu().item()

def train_model(model, num_epochs, trainloader, criterion, optimizer):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = model.to(device)

    losses = []
    accs = []

    norms = []

    for epoch in range(num_epochs):
        train_running_loss = 0.0
        train_acc = 0.0

        # Set the model to training mode
        model.train()
        ## training step
        for i, (inputs, labels) in enumerate(trainloader):

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            ## forward + backprop + loss
            logits = model(inputs)
            loss = criterion(logits, labels)

            # Reset the gradients to zero

            loss.backward()

            ## update model params
            optimizer.step()

            train_running_loss += loss.item()
            train_acc += get_accuracy(logits, labels)


        losses.append(train_running_loss / i)
        accs.append(train_acc/i)

        norms.append(compute_weight_norm(model))
        model.eval()
        print(f"Epoch: {epoch+1} | Loss: {train_running_loss / i:.4f} | Train Accuracy: {train_acc/i:.4f}")

    return losses, accs, norms

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(300, 1024)

        self.fc2 = nn.Linear(1024, 500)

        self.fc3 = nn.Linear(500, 256)

        self.fc4 = nn.Linear(256, 128)

        self.fc5 = nn.Linear(128, 64)

        self.fc6 = nn.Linear(64, 32)

        self.fc7 = nn.Linear(32, 8)

        self.fc8 = nn.Linear(8, 2)



    def forward(self, x):

        x = torch.relu(self.fc1(x))

        x = torch.relu(self.fc2(x))

        x = torch.relu(self.fc3(x))

        x = torch.relu(self.fc4(x))

        x = torch.relu(self.fc5(x))

        x = torch.relu(self.fc6(x))

        x = torch.relu(self.fc7(x))

        x = torch.sigmoid(self.fc8(x))
        return x


In [ ]:
input_size = X_train.shape[1]
output_size = 1
print(input_size)

model = MLP()

class_weights = torch.tensor([1.0, 5.0])

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1)



_, _, W_adam = train_model(model, 100, train_loader, criterion, optimizer)

In [ ]:



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

total = np.concatenate((data, data_cheater), axis=0)

labels_legit = np.zeros(len(data))
labels_cheater=np.ones(len(data_cheater))
labels = np.concatenate((labels_legit, labels_cheater))

X_train, X_test, y_train, y_test = train_test_split(total, labels, test_size=0.20, random_state=42, stratify=labels)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test_scaled = scaler.fit_transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

print(X_train_scaled.shape[1])
print(X_train_scaled.shape)

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(X_train_tensor.shape)
print(y_train_tensor.shape)

class MLP(nn.Module):
  def __init__(self):
        super(MLP, self).__init__()
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=64, kernel_size=(3,3,3), padding=1)
        self.conv2 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=(3,3,3), padding=1)
        self.pool = nn.MaxPool3d(kernel_size=(2,2,2), stride=2)
        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(43008, 256)
        self.fc2 = nn.Linear(256, 1)
        self.relu = nn.ReLU()
  def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


model = MLP()

class_weights = torch.tensor([1.0, 5.0])
criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights[1])
optimizer = optim.Adam(model.parameters(), lr=0.001)





In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs = inputs.unsqueeze(1)
        labels = labels.squeeze(1)
        optimizer.zero_grad()

        outputs = model(inputs).squeeze(1)
        loss = criterion(outputs, labels)


        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Valutare il modello
    model.eval()

    with torch.no_grad():
      correct = 0
      total = 0
      all_labels = []
      all_predicted = []
      for inputs, labels in test_loader:

        outputs = model(inputs)
        predicted = torch.sigmoid(outputs) > 0.5
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_labels.extend(labels.numpy())
        all_predicted.extend(predicted.numpy())

    accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy:.4f}')
    print(classification_report(all_labels, all_predicted, target_names=['legit', 'cheater']))

In [ ]:
#tesi

import numpy as np
import pandas as pd
import random as rnd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



#caricamento dati
data = np.load("/content/drive/My Drive/Colab Notebooks/legit.npy")
data_cheater = np.load("/content/drive/My Drive/Colab Notebooks/cheaters.npy")


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#calcolo medie e varianza per legit e cheater
medie = np.mean(data, axis=(1,2))
varianza = np.mean(data, axis=(1,2))
medie_cheaters = np.mean(data_cheater, axis=(1,2))
varianza_cheaters = np.mean(data_cheater, axis=(1,2))

#concatenazione
result=np.stack((medie,varianza), axis=1)
result_cheater=np.stack((medie_cheaters,varianza_cheaters), axis=1)
total = np.concatenate((data, data_cheater), axis=0)

labels_legit = np.zeros(len(result))
labels_cheater=np.ones(len(result_cheater))
labels = np.concatenate((labels_legit, labels_cheater))

#reshape dei dati per la SVM
X = total.reshape(12000, -1)
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.20, random_state=42, stratify=labels)

#rescaling dei dati
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

print(X_train_scaled.shape[1])
#modello svm linear
# svm = SVC(kernel = 'linear')
# svm.fit(X_train_scaled, y_train)
# #predicition
# y_pred=svm.predict(X_test_scaled)

#accuracy
# acc = accuracy_score(y_test, y_pred)
# print(acc)
# cm_lin = confusion_matrix(y_test, y_pred)
# disp_lin = ConfusionMatrixDisplay(confusion_matrix=cm_lin, display_labels=["Non cheater", "cheater"])
# disp_lin.plot()
# plt.show()

#modello svm gaussian
# svm_rbf = SVC(kernel = 'rbf', gamma=20)
# svm_rbf.fit(X_train_scaled, y_train)
# #predicition
# y_pred_rbf=svm_rbf.predict(X_test_scaled)

#accuracy
# acc_rbf = accuracy_score(y_test, y_pred_rbf)
# print(acc_rbf)
# cm_rbf = confusion_matrix(y_test, y_pred_rbf)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm_rbf, display_labels=["Non cheater", "cheater"])
# disp.plot()
# plt.show()


X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).view(-1,1,X_test_scaled.shape[1])
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).view(-1,1,X_test_scaled.shape[1])
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(X_train_tensor.shape)
print(y_train_tensor.shape)
def get_accuracy(logit, target):
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects / target.size(0)
    return accuracy.item()

def compute_weight_norm(model):
    norm = 0.0
    for name, param in model.named_parameters():
      if "weight" in name:
        norm += torch.norm(param.data, p= 2)
    return norm.cpu().item()

def train_model(model, num_epochs, trainloader, criterion, optimizer):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Rich mac user
    # device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

    model = model.to(device)

    losses = []
    accs = []

    norms = []

    for epoch in range(num_epochs):
        train_running_loss = 0.0
        train_acc = 0.0

        # Set the model to training mode
        model.train()
        ## training step
        for i, (inputs, labels) in enumerate(trainloader):

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            ## forward + backprop + loss
            logits = model(inputs)
            loss = criterion(logits, labels)

            # Reset the gradients to zero

            loss.backward()

            ## update model params
            optimizer.step()

            train_running_loss += loss.item()
            train_acc += get_accuracy(logits, labels)


        losses.append(train_running_loss / i)
        accs.append(train_acc/i)

        norms.append(compute_weight_norm(model))
        model.eval()
        print(f"Epoch: {epoch+1} | Loss: {train_running_loss / i:.4f} | Train Accuracy: {train_acc/i:.4f}")

    return losses, accs, norms

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=6, kernel_size=10)
        self.dropout1 = nn.Dropout(p=0.75)

        self.conv2 = nn.Conv1d(in_channels=6, out_channels=6, kernel_size=10)
        self.dropout2 = nn.Dropout(p=0.75)


        self.conv3 = nn.Conv1d(in_channels=6, out_channels=6, kernel_size=10)
        self.dropout3 = nn.Dropout(p=0.75)

        self.flattern = nn.Flatten()

        self.dropout4 = nn.Dropout(p=0.75)


        self.fc1 = nn.Linear(172638, 2)



    def forward(self, x):


        x = F.relu(self.conv1(x))
        x = self.dropout1(x)

        x = F.relu(self.conv2(x))
        x = self.dropout2(x)

        x = F.relu(self.conv3(x))
        x = self.dropout3(x)

        x = x.flatten(start_dim = 1)

        #x = self.dropout4

        x = torch.sigmoid(self.fc1(x))
        return x



input_size = X_train.shape[1]
output_size = 1

model = MLP()

# class_weights = torch.tensor([1.0, 5.0])
# criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

_, _, W_adam = train_model(model, 100, train_loader, criterion, optimizer)


# Addestrare il modello
# num_epochs = 100
# for epoch in range(num_epochs):
#     model.train()
#     for inputs, labels in train_loader:

#         optimizer.zero_grad()

#         outputs = model(inputs)
#         loss = criterion(outputs, labels)


#         loss.backward()
#         optimizer.step()

#     if (epoch+1) % 10 == 0:
#         print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# # Valutare il modello
#     model.eval()

#     with torch.no_grad():
#       correct = 0
#       total = 0
#       all_labels = []
#       all_predicted = []
#       for inputs, labels in test_loader:

#         outputs = model(inputs)
#         predicted = torch.sigmoid(outputs) > 0.5
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
#         all_labels.extend(labels.numpy())
#         all_predicted.extend(predicted.numpy())

#     accuracy = correct / total
#     print(f'Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy:.4f}')
#     print(classification_report(all_labels, all_predicted, target_names=['legit', 'cheater']))




